# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

***Инструкция по выполнению проекта***

* Разделите исходные данные на обучающую, валидационную и тестовую выборки.
* Исследуйте качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.
* Проверьте качество модели на тестовой выборке.
* Дополнительное задание: проверьте модели на вменяемость. Ничего страшного, если не получится: эти данные сложнее тех, с которыми вы работали раньше. В следующем курсе подробнее об этом расскажем.  

***Описание данных***  
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import numpy as np
from sklearn.dummy import DummyClassifier

df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Перед нами уже готовые данные, поэтому нет смысла смотреть подробную информацию о датафрейме.

## Разбейте данные на выборки

За целевой признак возьмём столбец 'is_ultra' и разобьём выборку по принципу 60% train (данные для обучения), 20% valid (для валидации модели), 20% test (для применения модели).

In [3]:
df_train, df_valid = train_test_split(df, test_size=0.4, random_state=12345)
df_valid,df_test = train_test_split(df_valid, test_size=0.5, random_state=12345)

In [4]:
# Тренировочные данные
features_train = df_train.drop('is_ultra', axis=1)
target_train = df_train['is_ultra']
# Валидационные данные
features_valid = df_valid.drop('is_ultra', axis=1)
target_valid = df_valid['is_ultra']
# Тестовые данные
features_test = df_test.drop('is_ultra', axis=1)
target_test = df_test['is_ultra']

print('Размер тренериующей выборки', df_train.shape[0])
print('Размер валидационной выборки', df_valid.shape[0])
print('Размер тестовой выборки', df_test.shape[0])

Размер тренериующей выборки 1928
Размер валидационной выборки 643
Размер тестовой выборки 643


## Исследуйте модели

### Исследуем модель Дерево принятия решения (с глубиной до 20 деревьев)

In [5]:
best_depth = 0
best_accuracy = 0
for depth in range(1, 21):
    dtc_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    dtc_model.fit(features_train, target_train)
    
    dtc_valid_predictions = dtc_model.predict(features_valid)
    accuracy = accuracy_score(target_valid, dtc_valid_predictions)
    print(f'max_depth = {depth}; accuracy = {accuracy}')
    if accuracy > best_accuracy:
        best_depth = depth
        best_accuracy = accuracy
print('Kоличество деревьев', best_depth,'Лучшая точность', best_accuracy) 

max_depth = 1; accuracy = 0.7542768273716952
max_depth = 2; accuracy = 0.7822706065318819
max_depth = 3; accuracy = 0.7853810264385692
max_depth = 4; accuracy = 0.7791601866251944
max_depth = 5; accuracy = 0.7791601866251944
max_depth = 6; accuracy = 0.7838258164852255
max_depth = 7; accuracy = 0.7822706065318819
max_depth = 8; accuracy = 0.7791601866251944
max_depth = 9; accuracy = 0.7822706065318819
max_depth = 10; accuracy = 0.7744945567651633
max_depth = 11; accuracy = 0.7620528771384136
max_depth = 12; accuracy = 0.7620528771384136
max_depth = 13; accuracy = 0.7558320373250389
max_depth = 14; accuracy = 0.7589424572317263
max_depth = 15; accuracy = 0.7465007776049767
max_depth = 16; accuracy = 0.7340590979782271
max_depth = 17; accuracy = 0.7356143079315708
max_depth = 18; accuracy = 0.7309486780715396
max_depth = 19; accuracy = 0.7278382581648523
max_depth = 20; accuracy = 0.7216174183514774
Kоличество деревьев 3 Лучшая точность 0.7853810264385692


Лучше всего модель себя проявляет при глубине дерева 3.

### Исследуем модель Случайный лес с количеством estimators от 1 до 20

In [6]:
best_estim = 0
best_accuracy = 0

for estim in range(1,21):
    model = RandomForestClassifier(random_state=12345, n_estimators=estim)
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)
    print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Kоличество деревьев', best_estim,'Лучшая точность', best_accuracy)        

Количество деревьев 1 Точность 0.7107309486780715
Количество деревьев 2 Точность 0.7636080870917574
Количество деревьев 3 Точность 0.7387247278382582
Количество деревьев 4 Точность 0.7713841368584758
Количество деревьев 5 Точность 0.749611197511664
Количество деревьев 6 Точность 0.7807153965785381
Количество деревьев 7 Точность 0.7682737169517885
Количество деревьев 8 Точность 0.7822706065318819
Количество деревьев 9 Точность 0.7729393468118196
Количество деревьев 10 Точность 0.7853810264385692
Количество деревьев 11 Точность 0.7838258164852255
Количество деревьев 12 Точность 0.7869362363919129
Количество деревьев 13 Точность 0.7822706065318819
Количество деревьев 14 Точность 0.7838258164852255
Количество деревьев 15 Точность 0.7838258164852255
Количество деревьев 16 Точность 0.7869362363919129
Количество деревьев 17 Точность 0.7869362363919129
Количество деревьев 18 Точность 0.7931570762052877
Количество деревьев 19 Точность 0.7884914463452566
Количество деревьев 20 Точность 0.7869362

Посмотрим, как поведёт себя модель при настройке гиперпараметров.

In [7]:
best_estim = 0
best_accuracy = 0

for estim in range(1,21):
    model = RandomForestClassifier(random_state=12345, n_estimators=18, criterion='entropy') # изменим criterion с gini на entropy
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)
    print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Kоличество деревьев', best_estim,'Лучшая точность', best_accuracy)        

Количество деревьев 1 Точность 0.7776049766718507
Количество деревьев 2 Точность 0.7776049766718507
Количество деревьев 3 Точность 0.7776049766718507
Количество деревьев 4 Точность 0.7776049766718507
Количество деревьев 5 Точность 0.7776049766718507
Количество деревьев 6 Точность 0.7776049766718507
Количество деревьев 7 Точность 0.7776049766718507
Количество деревьев 8 Точность 0.7776049766718507
Количество деревьев 9 Точность 0.7776049766718507
Количество деревьев 10 Точность 0.7776049766718507
Количество деревьев 11 Точность 0.7776049766718507
Количество деревьев 12 Точность 0.7776049766718507
Количество деревьев 13 Точность 0.7776049766718507
Количество деревьев 14 Точность 0.7776049766718507
Количество деревьев 15 Точность 0.7776049766718507
Количество деревьев 16 Точность 0.7776049766718507
Количество деревьев 17 Точность 0.7776049766718507
Количество деревьев 18 Точность 0.7776049766718507
Количество деревьев 19 Точность 0.7776049766718507
Количество деревьев 20 Точность 0.777604

In [8]:
best_estim = 0
best_accuracy = 0

for estim in range(1,21):
    model = RandomForestClassifier(random_state=12345, n_estimators=18, bootstrap=False) # изменим bootstrap с True на Flase
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)
    print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Kоличество деревьев', best_estim,'Лучшая точность', best_accuracy)        

Количество деревьев 1 Точность 0.7636080870917574
Количество деревьев 2 Точность 0.7636080870917574
Количество деревьев 3 Точность 0.7636080870917574
Количество деревьев 4 Точность 0.7636080870917574
Количество деревьев 5 Точность 0.7636080870917574
Количество деревьев 6 Точность 0.7636080870917574
Количество деревьев 7 Точность 0.7636080870917574
Количество деревьев 8 Точность 0.7636080870917574
Количество деревьев 9 Точность 0.7636080870917574
Количество деревьев 10 Точность 0.7636080870917574
Количество деревьев 11 Точность 0.7636080870917574
Количество деревьев 12 Точность 0.7636080870917574
Количество деревьев 13 Точность 0.7636080870917574
Количество деревьев 14 Точность 0.7636080870917574
Количество деревьев 15 Точность 0.7636080870917574
Количество деревьев 16 Точность 0.7636080870917574
Количество деревьев 17 Точность 0.7636080870917574
Количество деревьев 18 Точность 0.7636080870917574
Количество деревьев 19 Точность 0.7636080870917574
Количество деревьев 20 Точность 0.763608

In [9]:
best_estim = 0
best_accuracy = 0

for estim in range(1,21):
    model = RandomForestClassifier(random_state=12345, n_estimators=18, warm_start=True) # изменим warm_start с False на True
    model.fit(features_train, target_train)
    valid_predictions = model.predict(features_valid)
    accuracy = accuracy_score(target_valid, valid_predictions)
    print('Количество деревьев', estim,'Точность',accuracy)
    if accuracy > best_accuracy:
        best_estim = estim
        best_accuracy = accuracy

print('Kоличество деревьев', best_estim,'Лучшая точность', best_accuracy)        

Количество деревьев 1 Точность 0.7931570762052877
Количество деревьев 2 Точность 0.7931570762052877
Количество деревьев 3 Точность 0.7931570762052877
Количество деревьев 4 Точность 0.7931570762052877
Количество деревьев 5 Точность 0.7931570762052877
Количество деревьев 6 Точность 0.7931570762052877
Количество деревьев 7 Точность 0.7931570762052877
Количество деревьев 8 Точность 0.7931570762052877
Количество деревьев 9 Точность 0.7931570762052877
Количество деревьев 10 Точность 0.7931570762052877
Количество деревьев 11 Точность 0.7931570762052877
Количество деревьев 12 Точность 0.7931570762052877
Количество деревьев 13 Точность 0.7931570762052877
Количество деревьев 14 Точность 0.7931570762052877
Количество деревьев 15 Точность 0.7931570762052877
Количество деревьев 16 Точность 0.7931570762052877
Количество деревьев 17 Точность 0.7931570762052877
Количество деревьев 18 Точность 0.7931570762052877
Количество деревьев 19 Точность 0.7931570762052877
Количество деревьев 20 Точность 0.793157

### Логистическая регрессия

In [10]:
model_reg = LogisticRegression(random_state=12345,solver='lbfgs')
model_reg.fit(features_train,target_train ) 
result_reg = model_reg.score(features_valid,target_valid ) 
 
print("Точность модели логистической регрессии:", result_reg)

Точность модели логистической регрессии: 0.7107309486780715


### Вывод

Результаты по выборке:  

* DecisionTreeClassifier = max_depth = 3; accuracy = 0.7853810264385692  
* RandomForestClassifier = Kоличество деревьев 18 Лучшая точность 0.7931570762052877  
* LogisticRegression = Точность модели логистической регрессии: 0.7107309486780715  

Наиболее высокий результат на валидационной выборке показали:  

* RandomForestClassifier = Kоличество деревьев 18 Лучшая точность 0.7931570762052877   

При настройке гиперпараметров entropy и bootstrap=False результаты стали хуже, при изменении warm_start с False на True результат не изменился.

## Проверьте модель на тестовой выборке

In [11]:
model = RandomForestClassifier(random_state=12345, n_estimators=18, warm_start=True)
model.fit(features_train, target_train)
predictions = model.predict(features_test)

print(f"RandomForestClassifier. n_estimators = 18; accuracy = {accuracy_score(target_test, predictions)}")

RandomForestClassifier. n_estimators = 18; accuracy = 0.7853810264385692


In [12]:
test_predictions = dtc_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели решающего леса на тестовой выборке',accuracy)

test_predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

test_predictions = model_reg.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке',accuracy)

Точность модели решающего леса на тестовой выборке 0.7325038880248833
Точность модели случайного леса на тестовой выборке 0.7853810264385692
Точность модели логистической регресиии на тестовой выборке 0.6842923794712286


Для построения модели для задачи классификации, которая выберет подходящий тариф, будем использовать модель случайного леса с точностью на тестовой выборке 78.5%.

## (бонус) Проверьте модели на адекватность

In [13]:
test_predictions = model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
precision = precision_score(target_test, test_predictions)
recall = recall_score(target_test, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

Accuracy = 0.7853810264385692 Precision = 0.7018633540372671 Recall = 0.5566502463054187


Для оценки адеватности используется F-мера - среднее гармоническое между precision и recall. 

In [14]:
f_score = f1_score(target_test, test_predictions)
print('F-мера =', f_score)

F-мера = 0.6208791208791209


In [15]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

In [16]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_test, target_test)
DummyClassifier(strategy='most_frequent')
dummy_clf.predict(features_test)
dummy_clf.score(features_test, target_test)

0.6842923794712286

## Вывод

Учитывая небольшую выборку модель получилась неплохая. Нам удалось добиться accuracy 78.5%. С увеличением количества данных будет расти и обучаемость. Для построения модели для задачи классификации, которая выберет подходящий тариф, мы использовали модель случайного леса с точностью на тестовой выборке 78.5%.  
Для проверки модели на адекватность мы взяли несколько методов: F-меру (62%), константу - относительную частоту уникальных значений (69%) и классификатор (68%). Наша модель в трёх проверках оказалась лучше, что говорит о её предсказательных способностях.